In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import ast 
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
import time
import random

In [0]:
import pandas as pd
path = "/content/drive/My Drive/spacy_train_clean.csv"
train_df = pd.read_csv(path)
train_df['entities_clean']=[ast.literal_eval(i) for i in train_df['entities_clean']]


In [0]:
def create_train_data(df):
  train_data = []
  newnlp = spacy.load("en_core_web_sm")

  for i in range(1000):
    doc = newnlp(train_df['text'].iloc[i])
    entity_list = train_df['entities_clean'][i]
    for ent in doc.ents:
      entity_list.append((ent.start_char, ent.end_char, ent.label_))
    entity_dict = {"entities": entity_list}
    train_data.append((df['text'][i], entity_dict))
  return train_data

In [0]:
def create_test_data(df):
  train_data = []
  newnlp = spacy.load("en_core_web_sm")

  for i in range(200):
    doc = newnlp(train_df['text'].iloc[2000+i])
    entity_list = train_df['entities_clean'][2000+i]
    for ent in doc.ents:
      entity_list.append((ent.start_char, ent.end_char, ent.label_))
    entity_dict = {"entities": entity_list}
    train_data.append((df['text'][2000+i], entity_dict))
  return train_data

In [7]:
TRAIN_DATA

[("this is my second visit; my first was back during ces in jan. with a young friend visiting vegas for the first time and who had never had a bloody mary...\n\nservice was very good, the large bloody mary was impressive, and the jack daniel's chocolate cake was to die for.\n\ni liked it enough to want to go again.\n\nexcept for the big bloody mary meal - slim jim, pickles, cheese cubes, peppers,  candied bacon !!!!; sausage; red onion (see pics, only large u6 shrimp were missing. ..) everything else was a disappointment.\n\nservice was very strange - the smiles were abundant, but everything else about the service was off, from how scarce our server was, to her not willing to take an order for breakfast from us,  yet a table seated after us served by someone else got breakfast; to how long it took to place our orders, to how long it took to get our order.\n\ni got the wings - terrible, even though they are a signature dish. bland, small, bones broken with sharp points and bone chips al

In [6]:
# new entity label
LABEL = "PRODUCT"

TRAIN_DATA = create_train_data(train_df)
TEST_DATA = create_test_data(train_df)

def main(model='en_core_web_sm', new_model_name="product", output_dir=None, n_iter=15):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")

    ner.add_label(LABEL)  # add new entity label to entity recognizer
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(1.0, 4.0, 1.001)
        # batch the examples using spaCy's minibatch
        start = time.time()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)            
            #print("Training Recall:",nlp.evaluate(random.sample(TRAIN_DATA,200)).ents_r)
            print("Test Recall:",nlp.evaluate(TEST_DATA).ents_p)
            #print("Training Losses", losses)
        end = time.time()
    print("Total training time:",end-start)

    # test the trained model
    for i in range(10):
      test_text = train_df['text'][2000+i]
      doc = nlp(test_text)
      print("Entities in '%s'" % test_text)
      for ent in doc.ents:
          print(ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        # Check the classes have loaded back consistently
        assert nlp2.get_pipe("ner").move_names == move_names
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)
    return nlp

model = main()

Loaded model 'en_core_web_sm'
Test Recall: 90.79528718703976
Test Recall: 91.7948717948718
Test Recall: 89.93205682520075
Test Recall: 91.49064574532288
Test Recall: 90.66745422327229
Test Recall: 89.84238178633976
Test Recall: 89.4828588030215
Test Recall: 92.01409277745157
Test Recall: 91.11508646392367
Test Recall: 91.51943462897526
Test Recall: 90.73756432246999
Test Recall: 91.44698301113064
Test Recall: 91.76267281105991
Test Recall: 91.83555298204979
Test Recall: 90.34123770965876
Total training time: 543.3941941261292
Entities in 'based on the nam eighties place i would expect otto be bar style food. however, their menu is a little fancy compared to what i had expected. their salads are very good and the service has always been great. there's not a lot of parking close to it but at least it's located in a shopping center. wait times are sometimes very long and there aren't many seats for people who are waiting. there is also a massive bird problem if you happen to sit outside. 

In [0]:
test_text = train_df['text'][26]
doc = model(test_text)
#print("Entities in '%s'" % test_text)
for ent in doc.ents:
    print(ent.start_char, ent.end_char, ent.label_)
for ent in doc.ents:
    print(ent.label_, ent.text)

(must, 10+ years, special, 30 minutes)


AttributeError: ignored

In [0]:
test_text = train_df['text'][26]
newdoc = newnlp(test_text)
for ent in newdoc.ents:
    print(ent.start_char, ent.end_char, ent.label_)

374 383 DATE
424 433 DATE
525 535 TIME


In [0]:
train_df['text'][26]

'unfortunately, i must recommend not to conduct any business with fast fix jewelry and watch repairs.  the staff is completely unprofessional and i have never been treated so rudely in my life.  i was actually shocked and walked away thinking "wow, how do those people even have a job?"\n\ni have never written a negative review before, i usually chalk it up to someone having a bad day.  i have worked in customer service for 10+ years and i know it is exhausting and bad days happen.  but this is a special occasion.\n\ni drove 30 minutes to try to find someone who could help me repair a watch band.  i was barely greeted upon arrival and right away felt like i was bothering them.  the girl behind the counter obviously just wanted me to leave, the blond kid working snapped at me when i asked a question and when i re-asked in order to clarify what he said he snapped again and said, "i already told you?!?"  "is this even authentic?"  shouting at me over his shoulder as he work on something el

In [0]:
brand_list = []
for (n,i) in enumerate(train_df['entities_clean'][:100]):
  text = train_df['text'][n]
  for pair in i:
    brand_list.append(text[pair[0]:pair[1]])
    
import numpy as np
unique_brands = np.unique(brand_list)

In [0]:
import numpy as np

new_brand_list = []
for (n,i) in enumerate(train_df['text'][:100]):
  text = train_df['text'][n]
  doc = model(text)
  for ent in doc.ents:
    new_brand_list.append(ent.text)
new_unique_brands = np.unique(new_brand_list)

In [0]:
type(unique_brands[0])

numpy.str_

In [0]:
type(new_unique_brands[0])

numpy.str_

In [0]:
in_common = list(set(unique_brands) & set(new_unique_brands))

In [0]:
in_common

['bun',
 'quesadilla',
 'ravioli',
 'spice',
 'cole',
 'coconut',
 'eggplant',
 'calamari',
 'formula',
 'entree',
 'veal',
 'mayo',
 'spread',
 'plate',
 'carbonara',
 'garlic',
 'butter',
 'appetizer',
 'lunch',
 'burger',
 'potato',
 'hamburger',
 'soda',
 'salmon',
 'ham',
 'wonton',
 'charlotte',
 'pepperoni',
 'sashimi',
 'wine',
 'beer',
 'tortilla',
 'diet',
 'taco',
 'bit',
 'rib',
 'noodle',
 'must',
 'meal',
 'corn',
 'sandwich',
 'stout',
 'date',
 'blueberry',
 'catfish',
 'flip',
 'baguette',
 'parmesan',
 'tapioca',
 'batter',
 'tofu',
 'beef',
 'table',
 'rock',
 'bourbon',
 'special',
 'water',
 'salsa',
 'sucker',
 'dog',
 'coffee',
 'game',
 'milkshake',
 'toast',
 'board',
 'brownie',
 'burrito',
 'sweet',
 'crab',
 'veggie',
 'tater',
 'vinaigrette',
 'yogurt',
 'lamb',
 'strawberry',
 'tea',
 'biryani',
 'cut',
 'duck',
 'delicious',
 'portion',
 'menu',
 'salad',
 'cheese',
 'scampi',
 'mustard',
 'pizza',
 'popcorn',
 'milk',
 'pasta',
 'naan',
 'rice',
 'dinner

In [0]:
len(unique_brands)

222

In [0]:
new_unique_brands

array(['&', '0.50', '1', '10', '10 minutes', '10 years', '10+ years',
       '100', '100.00', '12', '14', '15', '15+ years', '17', '19 cents',
       '1:30 pm', '1:45 p.m.', '2', '20', '20 minutes', '200', '25',
       '25 minutes', '25%', '280', '3', '3 years', '3-$4', '30 minute',
       '30 minutes', '35th', '37', '3:04 pm', '3:30', '4', '45', '5',
       '5 minutes ago', '5.25', '50.00', '51 february 5, 2017', '6', '66',
       '69', '6:30 p.m.', '7', '7-20', '72', '8', '80', '9', '92',
       'a couple months', 'a couple years ago', 'a few days',
       'a few days ago', 'a few days later', 'a few hours',
       'a few minutes', 'a few times years ago', 'a few years ago',
       "a mid 80's", 'a minute', 'a month', 'a night',
       'a two hour period', 'a week', 'a year', 'about 10 minutes',
       'about 15 minutes', 'about 30 seconds', 'about 5', 'about 50',
       'about two years ago', 'afternoon', 'alcohol', 'all morning',
       'all night', 'almost $4', 'an extra $5', 'ano

In [0]:
np.setdiff1d(unique_brands,new_unique_brands)

array(['apple', 'apple pie', 'asparagus', 'avocado', 'bloody mary',
       'breakfast food', 'caesar salad', 'center', 'cheese dip',
       'cheeseburger', 'cheesecake', 'chilli', 'chipotle', 'chuck',
       'coleslaw', 'comfort food', 'congee', 'corn beef', 'course',
       'crab legs', 'cream', 'cut of meat', 'dip', 'dumplings',
       'fast food', 'fixings', 'frozen yogurt', 'fudge', 'fudge sauce',
       'gem', 'goat cheese', 'grass', 'grits', 'hash', 'honeydew',
       'horseradish', 'hot dog', 'hot pot', 'jam', 'jelly', 'jerk',
       'joint', 'kale', 'lime', 'olive', 'onion', 'orange', 'penne',
       'pie', 'pork belly', 'potato salad', 'pudding', 'pumpkin',
       'red meat', 'red wine', 'riesling', 'rose', 'sangria', 'sausage',
       'savory', 'sea bass', 'serving', 'short ribs', 'shoulder', 'slice',
       'snack', 'sour', 'sugar', 'sugar cookie', 'sweet potato', 'taro',
       'tongue', 'topping', 'vanilla', 'vegetable', 'vodka', 'wasabi',
       'white chocolate', 'wine s

In [0]:
np.setdiff1d(new_unique_brands,unique_brands)

array(['&', '0.50', '1', '10', '10 minutes', '10 years', '10+ years',
       '100', '100.00', '12', '14', '15', '15+ years', '17', '19 cents',
       '1:30 pm', '1:45 p.m.', '2', '20', '20 minutes', '200', '25',
       '25 minutes', '25%', '280', '3', '3 years', '3-$4', '30 minute',
       '30 minutes', '35th', '37', '3:04 pm', '3:30', '4', '45', '5',
       '5 minutes ago', '5.25', '50.00', '51 february 5, 2017', '6', '66',
       '69', '6:30 p.m.', '7', '7-20', '72', '8', '80', '9', '92',
       'a couple months', 'a couple years ago', 'a few days',
       'a few days ago', 'a few days later', 'a few hours',
       'a few minutes', 'a few times years ago', 'a few years ago',
       "a mid 80's", 'a minute', 'a month', 'a night',
       'a two hour period', 'a week', 'a year', 'about 10 minutes',
       'about 15 minutes', 'about 30 seconds', 'about 5', 'about 50',
       'about two years ago', 'afternoon', 'all morning', 'all night',
       'almost $4', 'an extra $5', 'another 5 minu